In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import cm
import os
import imageio
from PIL import Image
from scipy import stats
from pathlib import Path


sample_name = "future_work"
img_dest = f"images/{sample_name}"
out_dir = f"../../output/distributed_model/{sample_name}/"
series_dir = f"{out_dir}cov_series/"
series_files = os.listdir(series_dir)

geo_filtered = pd.read_csv('../../../datasets/mobility/Geolife_Trajectories_1.3/Data/geolife_filtered.csv')
pois_filtered = pd.read_csv('../../input/pois_filtered.csv')
cov = pd.read_csv(f'{out_dir}coverage.csv').drop(columns=["Unnamed: 0"])

Path(f"{img_dest}/").mkdir(parents=True, exist_ok=True)


cov_series = []
if '.ipynb_checkpoints' in series_files:
    series_files.remove('.ipynb_checkpoints')
#print(series_files)

for file in series_files:
    #print(series_dir + file)
    df = pd.read_csv(series_dir + file).rename(columns={"Unnamed: 0": "id_location"})
    node = file.split("_")[0]
    df['node'] = node
    cov_series.append(df)
    
cov_series = pd.concat(cov_series)
cov_series = cov_series.join(pois_filtered.set_index('id_location'), on="id_location", how="left")
cov = cov.join(pois_filtered.set_index('id_location'), on="id_location", how="left")

merges = pd.read_csv(f"{out_dir}log_merge.csv")

In [4]:
merges[merges.node_1 == 82]#['node_2'].unique()

,node_1,node_2,bid


In [5]:
nodes = ['82', '84', '180', '153']


for node in nodes:
    Path(f"{img_dest}/series/{node}/3D/").mkdir(parents=True, exist_ok=True)
    Path(f"{img_dest}/series/{node}/heatmap/").mkdir(parents=True, exist_ok=True)

    node_serie = cov_series[cov_series.node == node]

    numcols, numrows = 100, 100
    print(img_dest)


    count = 1 
    for c in node_serie.copy_id.unique():
        data = node_serie[node_serie.copy_id == c]

        fig = plt.figure(figsize=(15,15))
        ax = fig.gca(projection='3d')

        xi = np.linspace(data.lon.min(), data.lon.max(), numrows)
        yi = np.linspace(data.lat.min(), data.lat.max(), numcols)

        #print(xi, yi)

        xi, yi = np.meshgrid(xi, yi)


        x, y, z = data.lon, data.lat, data.probability


        # known points (long, lat)
        points = np.vstack((x,y)).T 
        # known z points
        values = z
        # all the points I want to interpolate
        wanted = (xi, yi)
        # meshgrid interpolated
        zi = griddata(points, values, wanted,method="linear", fill_value = 0) 

        surf = ax.plot_surface(xi, yi, zi, cmap=cm.Reds,
                               linewidth=0, antialiased=False,shade=True)

        ax.set_zlim(0, 1)
        #ax.zaxis.set_major_locator(LinearLocator(10))
        #ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

        # Add a color bar which maps values to colors.
        #fig.colorbar(surf, shrink=0.3, aspect=5)

        bid = node_serie[node_serie.copy_id == c].bid.unique()[0]
        print(bid)
        #plt.show()
        plt.ioff()
        fig = plt.gcf()
        fig.set_size_inches(7, 7)
        plt.title(f'{bid}')
        plt.tight_layout()
        plt.savefig(f'{img_dest}/series/{node}/3D/{count}_{node}_3D_grid.png', dpi  = 100)
        plt.clf()
        plt.close()

        fig = plt.gcf()
        fig.set_size_inches(7, 7)
        plt.contourf(xi, yi, zi)
        plt.scatter(data.lon, data.lat, c=data.probability, s=60, vmin=zi.min(), vmax=zi.max(),cmap=cm.Reds)
        plt.ticklabel_format(useOffset=False)
        plt.colorbar()
        plt.title(f'{bid}')
        plt.tight_layout()
        plt.savefig(f"{img_dest}/series/{node}/heatmap/{count}_{node}_heatmap.png", dpi  = 300)

        plt.close()
        count += 1



images/future_work
2009-05-04 01:23:00


C:\Users\alexander\AppData\Local\Temp\ipykernel_16356\3230029057.py:19: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')


2009-05-05 14:08:00
2009-05-06 01:25:00
2009-05-06 11:41:00
2009-05-07 01:02:00
2009-05-10 13:12:00
2009-05-11 01:14:00
2009-05-13 11:57:00
2009-05-15 12:19:00
2009-05-31 19:24:00
images/future_work
2009-05-03 14:52:00
2009-05-04 10:45:00
2009-05-05 11:50:00
2009-05-05 22:53:00
2009-05-06 15:53:00
2009-05-06 22:43:00
2009-05-08 01:20:00
2009-05-09 09:32:00
2009-05-10 04:07:00
2009-05-17 06:10:00
2009-05-17 22:53:00
2009-05-18 09:52:00
2009-05-19 00:13:00
2009-05-23 01:40:00
2009-05-24 03:44:00
2009-05-25 14:20:00
2009-05-29 05:23:00
2009-05-29 13:24:00
2009-05-31 01:27:00
2009-05-31 19:24:00
images/future_work
2009-05-07 14:52:00
2009-05-31 19:24:00
images/future_work
2009-05-10 04:59:00
2009-05-10 12:26:00
2009-05-16 02:05:00
2009-05-16 07:03:00
2009-05-24 02:24:00
2009-05-24 10:09:00
2009-05-27 04:26:00
2009-05-28 03:37:00
2009-05-28 09:56:00
2009-05-31 19:24:00
